In [ ]:
!ls ../data/blast/

In [ ]:
path_to_fasta = "../data/blast/sequences.fasta"

In [ ]:
import os

In [ ]:
with open(path_to_fasta,"r") as fasta_file:
    if os.path.isfile("../data/blast/clean_sequences.fasta") == True:
        print("[-] File Already Exists!")
    else:
        print("[+] File Does Not Exist!")
        with open("../data/blast/clean_sequences.fasta","w") as clean_file:
            for line in fasta_file.readlines():
                if line != "\n":
                    clean_file.write(line)

In [ ]:
!mkdir ../results/blast/

In [ ]:
%%bash

blastn -db core_nt -outfmt "6 qseqid sseqid pident evalue bitscore qgi sgi sacc staxids sscinames scomnames stitle slen" \
-evalue 0.005 -max_target_seqs 10 -query ../data/blast/clean_sequences.fasta -out ../results/blast/blast_output.table -remote 

In [ ]:
import pandas as pd

In [ ]:
blast_table = pd.read_table("../results/blast/blast_output.table", header=None)
blast_table.columns = "qseqid sseqid pident evalue bitscore qgi sgi sacc staxids sscinames scomnames stitle slen".split(" ")
blast_table.head()

In [ ]:
co1_ids = ["JRA287","JRA284","JRA290","JRA281","JRA297","JRA296","JRA263","JRA268","JRA257","JRA260","JRA251","JRA254","JRA277","JRA276","JRA269","JRA272"]

len(co1_ids)

In [ ]:
id_to_taxonomy = {}
for query in blast_table.qseqid.unique():
    query_id = query.split("_")[0]
    if query_id in co1_ids:
        temp_table = blast_table[blast_table["qseqid"] == query]
        temp_table = temp_table.sort_values(by="pident", ascending=False)
        id_to_taxonomy[query_id] = " ".join(temp_table["stitle"].to_list()[0].split(" ")[:2])
#id_to_taxonomy

In [ ]:
with open("../data/blast/clean_sequences.fasta","r") as cleaned_sequences:
    with open("../data/blast/co1_sequences.fasta","w") as co1_sequences:
        for line in cleaned_sequences.readlines():

            if line.startswith(">"):
                header_id = line.rstrip()[1:].split("_")[0]
                if header_id in co1_ids:
                    switch = True
                    line = ">" + header_id + id_to_taxonomy[header_id].split(" ")[1] + "\n"
                    #print(line.rstrip())
                else:
                    switch = False 

            if switch == True:
                co1_sequences.write(line)

In [ ]:
!mafft ../data/blast/co1_sequences.fasta > ../results/co1_sequences.msa

In [ ]:
!fasttree -nt ../results/co1_sequences.msa > ../results/co1_sequences.nwk